In [23]:
# Data Sim • Noah González · 2025
# --------------------------------------------------
import pandas as pd
import numpy as np
from datetime import date

np.random.seed(42)              

# Params
retailers = ['LIVERPOOL', 'WALMART', 'PALACIO', 'COPPEL', 'SAMS']     
models    = ['OLED55C5PSA', 'OLED65C5PSA', 'OLED77C5PSA',
             '55QNED85APA', '65QNED85APA']                             

# weeks 2025-W23 … 2025-W45 (inclusive)
iso_weeks = [f"2025-W{w:02d}" for w in range(23, 46)]


# PSI DF
rows_psi = []
for acc in retailers:
    for mod in models:
        serie = 'C5' if 'OLED' in mod else 'QNED85'
        for wk in iso_weeks:
            wk_num = int(wk.split('-W')[1])
            display_val  = np.random.randint(15, 151)  if wk_num <= 26 else np.nan
            inv_sell = np.random.randint(50, 251)  if wk_num <= 26 else np.nan
            fcst     = np.random.randint(5, 101)    
            rows_psi.append((acc, mod, serie, wk, display_val, inv_sell, fcst))

psi_df = pd.DataFrame(rows_psi,
                      columns=['Account', 'Modelo', 'Serie', 'Semana',
                               'Display', 'Inventory_Sellable', 'Sell_Out_Forecast'])


# Target DF (monthly)
months_iso = ['2025-06', '2025-07', '2025-08', '2025-09']   

rows_tgt = [(acc, mod, mes, np.random.randint(15, 151))
            for acc in retailers
            for mod in models
            for mes in months_iso]

target_df = pd.DataFrame(rows_tgt,
                         columns=['Account', 'Modelo', 'Mes', 'Target'])


# Calendar DF
cal_rows = []
for wk in iso_weeks:
    yr, wk_num = 2025, int(wk.split('-W')[1])
    dt = date.fromisocalendar(yr, wk_num, 1)         
    cal_rows.append((wk, dt.strftime("%Y-%m")))

calendar_df = pd.DataFrame(cal_rows, columns=['Semana', 'Mes'])


# psi_df.to_csv('psi_base.csv', index=False)
# target_df.to_csv('targets_mensuales.csv', index=False)
# calendar_df.to_csv('calendario_semanas.csv', index=False)


In [24]:
weekly_tgt = (target_df.merge(calendar_df, on="Mes", how="left").loc[:, ["Account", "Modelo", "Semana", "Target"]].astype({"Semana": "string"}))

In [25]:
#  MERGE CON PSI

psi_df["Semana"]      = psi_df["Semana"].astype("string")
weekly_tgt["Semana"]  = weekly_tgt["Semana"].astype("string")

merged_df = (psi_df.merge(weekly_tgt, on=["Account", "Modelo", "Semana"], how="left").sort_values(["Account", "Modelo", "Semana"]))

In [26]:
#  % Achievement
merged_df["%Achievement"] = (merged_df["Display"] / merged_df["Target"]).round(3)

In [27]:
merged_df = merged_df.copy()  # evita SettingWithCopy
merged_df["Fecha_semana"] = pd.to_datetime(merged_df["Semana"] + "-1",format="%G-W%V-%u")

merged_df = (merged_df.sort_values(["Account", "Modelo", "Fecha_semana"]).reset_index(drop=True))

In [28]:
def avg_next_8_weeks(series, window=8):
    shifts = [series.shift(-k) for k in range(1, window + 1)]
    mat    = pd.concat(shifts, axis=1)
    valid  = mat.notna().all(axis=1)         
    avg    = mat.mean(axis=1, skipna=False)  

    return avg.where(valid)                   # respeta min. 8 semanas

merged_df["Avg_SO_FCST_next_8"] = (merged_df.groupby(["Account", "Modelo"], group_keys=False)["Sell_Out_Forecast"].apply(avg_next_8_weeks))

merged_df["WOS+8"] = (merged_df["Inventory_Sellable"] / merged_df["Avg_SO_FCST_next_8"]).where(merged_df["Inventory_Sellable"].notna()).round(1)


merged_df["Flooring Warning"]   = (merged_df["Inventory_Sellable"] >= (merged_df["Target"] - merged_df["Display"])).astype(int)
merged_df["Inventory Warning"] = (merged_df["WOS+8"] < 4).astype(int)

In [29]:
merged_df.columns

Index(['Account', 'Modelo', 'Serie', 'Semana', 'Display', 'Inventory_Sellable',
       'Sell_Out_Forecast', 'Target', '%Achievement', 'Fecha_semana',
       'Avg_SO_FCST_next_8', 'WOS+8', 'Flooring Warning', 'Inventory Warning'],
      dtype='object')

In [30]:
df_final = merged_df[['Account', 'Modelo', 'Serie', 'Semana', 'Inventory_Sellable', 'Display', 'Target', '%Achievement', 'Flooring Warning', 'WOS+8', 'Inventory Warning']]

In [44]:
from dash import Dash, dcc, html, dash_table, Input, Output, ctx
import dash_bootstrap_components as dbc
import pandas as pd
from dash_table.Format import Format, Scheme, Symbol
from dash_table import FormatTemplate

LG_RED = "#A50034" 


df_final = df_final.copy()
df_final["Flooring Warning"]  = df_final["Flooring Warning"].astype(int)
df_final["Inventory Warning"] = df_final["Inventory Warning"].astype(int)

# ════════════════════════
# Dash app
# ════════════════════════
app = Dash(__name__,
           external_stylesheets=[dbc.themes.BOOTSTRAP],
           suppress_callback_exceptions=True)

def make_opts(series):
    return [{"label": v, "value": v} for v in sorted(series.dropna().unique())]

app.layout = dbc.Container([
    html.H2("Flooring & Inventory Followup - Test",
            style={"backgroundColor": LG_RED, "color": "white",
                   "textAlign": "center", "padding": "12px",
                   "borderRadius": "6px", "marginTop": "10px"}),
    
    dbc.Row([
        dbc.Col(dcc.Dropdown(id="f-account",
                             multi=True,
                             options=make_opts(df_final["Account"]),
                             placeholder="Cuenta")),
        dbc.Col(dcc.Dropdown(id="f-serie",
                             multi=True,
                             options=make_opts(df_final["Serie"]),
                             placeholder="Serie")),
        dbc.Col(dcc.Dropdown(id="f-modelo",
                             multi=True,
                             options=make_opts(df_final["Modelo"]),
                             placeholder="Modelo")),
        dbc.Col(dcc.Dropdown(id="f-semana",
                             multi=True,
                             options=make_opts(df_final["Semana"]),
                             placeholder="Semana")),
    ], className="my-3"),
    
    dash_table.DataTable(
        id="tabla",
        #page_action="none",
        sort_action="native",
        fixed_rows={"headers": True},
        page_size=100,                   
        virtualization=True,            
        style_table={"height": "calc(100vh - 220px)", "overflowY": "auto"},
        style_cell={"textAlign": "center"},
        # ▸ Encabezados rojo LG
        style_header={"backgroundColor": LG_RED,
                      "color": "white",
                      "fontWeight": "bold",
                      "textAlign": "center"},
        # ▸ Formato de columnas
        columns=[
            {"name": col, "id": col,
             "type": "numeric",
             "format": FormatTemplate.percentage(1)} #Format(precision=1, scheme=Scheme.percentage, symbol=Symbol.yes)}
            if col == "%Achievement" else
            {"name": col, "id": col}
            for col in df_final.columns
        ],
        
        style_data_conditional=[
            
            {"if": {"filter_query": "{Flooring Warning} = 1",
                    "column_id": "Flooring Warning"},
             "backgroundColor": "#f8d7da"},
            {"if": {"filter_query": "{Inventory Warning} = 1",
                    "column_id": "Inventory Warning"},
             "backgroundColor": "#f8d7da"},
            
            {"if": {"filter_query": "{%Achievement} < 0.9",
                    "column_id": "%Achievement"},
             "backgroundColor": "#f8d7da"},
        ],
    )
], fluid=True)


# Callbacks

@app.callback(
    Output("f-modelo", "options"),
    Input("f-serie", "value"))
def actualiza_modelos(series_sel):
    if not series_sel:
        return make_opts(df_final["Modelo"])
    if isinstance(series_sel, str):
        series_sel = [series_sel]
    subset = df_final[df_final["Serie"].isin(series_sel)]
    return make_opts(subset["Modelo"])

@app.callback(
    Output("tabla", "data"),
    [Input("f-account", "value"),
     Input("f-serie",   "value"),
     Input("f-modelo",  "value"),
     Input("f-semana",  "value")]
)
def filtra_tabla(accounts, series, modelos, semanas):
    dff = df_final.copy()
    if accounts:
        dff = dff[dff["Account"].isin(accounts)]
    if series:
        dff = dff[dff["Serie"].isin(series)]
    if modelos:
        dff = dff[dff["Modelo"].isin(modelos)]
    if semanas:
        dff = dff[dff["Semana"].isin(semanas)]
    return dff.to_dict("records")


app.run(debug=True, port=8050, jupyter_mode="external")


Dash app running on http://127.0.0.1:8050/
